Tools  
[Write Midi Files](https://github.com/MarkCWirt/MIDIUtil)  
[Midi to CSV](https://pypi.org/project/py-midicsv/)  
[ABC Notation Player](https://github.com/mdoege/PySynth) 
[Midi to Wav](https://pypi.org/project/midi2audio/)  
[Mido <-> ABC in C](https://github.com/leesavide/abcmidi)  
[General Midi Instruments](https://en.wikipedia.org/wiki/General_MIDI#Bass)  


Resources  
[Andrej C-RNN Blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)  
[Midi File Format Info](https://www.csie.ntu.edu.tw/~r92092/ref/midi/)  
[Audio Stype Transfer Using Autoencoder](https://arxiv.org/abs/1812.07159)  



Architectures  
[C-RNN](https://github.com/karpathy/char-rnn)
[Autoencoder Example](https://www.learnopencv.com/understanding-autoencoders-using-tensorflow-python/)  


Tutorials  
[Siraj Uses Magenta](https://www.youtube.com/watch?v=pg9apmwf7og)  
[Toward Data Science Article](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)  

MIDI Library  
[Cprato](https://www.cprato.com)  
[Midi File Library](https://bitmidi.com)  

Notes  
CaryKH added Mozart to Bach, which doubles sample size. He then tranposed all data 6 times, one for each semitone. Then modifed Adnrej's network to 3 layers and 700 neurons each. Transposing the music up a half step makes it look like a new music (in pitch) to the network [Link](https://www.youtube.com/watch?v=SacogDL_4JU)




In [44]:
# Tools
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import trange
import glob
import pickle

# Play wav files
import IPython.display as ipd

# Play Midi Files

In [2]:
import pygame

def play_music1(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print ("Music file %s loaded!" % music_file)
    except pygame.error:
        print ("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
        
def play_music(music_file):
    # Play Midi file
    try:
        play_music1(music_file)
    except KeyboardInterrupt:
        # if user hits Ctrl/C then exit
        # (works only in console mode)
        pygame.mixer.music.fadeout(1000)
        pygame.mixer.music.stop()
        raise SystemExit
        
def play_music_csv(music_csv):        
    # Parse the CSV output of the previous command back into a MIDI file
    midi_object = py_midicsv.csv_to_midi(music_csv)

    # Save the parsed MIDI file to disk
    with open("example_converted.mid", "wb") as output_file:
        midi_writer = py_midicsv.FileWriter(output_file)
        midi_writer.write(midi_object)
    play_music("example_converted.mid")

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## Run this code of midi player doesn't work

In [5]:
# Run this code of midi player doesn't work

midi_file = 'data/midi/'+df_edm['filenames'][0]
freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)

# optional volume 0 to 1.0
pygame.mixer.music.set_volume(0.8)
try:
    play_music(midi_file)
except KeyboardInterrupt:
    # if user hits Ctrl/C then exit
    # (works only in console mode)
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit

Music file data/midi/Alan Walker - Alone  (midi by Carlo Prato) (www.cprato.com).mid loaded!


SystemExit: 

/Users/matthewlee/anaconda3/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Midi <-> CSV

## Compile data in pandas dataframe

In [3]:
import py_midicsv
import os

# Load file names
filenames = os.listdir('data/midi')
if '.DS_Store' in filenames:
    filenames.remove('.DS_Store')

filenames_clean = filenames.copy()
# Remove Unnecessary Strings
for ix,file in enumerate(filenames_clean):
    filenames_clean[ix] = filenames_clean[ix].replace('  (midi by Carlo Prato) (www.cprato.com)','')
    filenames_clean[ix] = filenames_clean[ix].replace(' (midi by Carlo Prato) (www.cprato.com)','')

# Generate Pandas Dataframe with filenames and CSV version of MIDI files
csvs = []
music_type = []
for filename in filenames:
    csvs.append(py_midicsv.midi_to_csv('data/midi/'+filename))
df_edm = pd.DataFrame({'filenames':filenames,'filenames_clean':filenames_clean,'csvs':csvs})
df_edm['filenames_clean2'] = df_edm['filenames_clean'].copy()
df_edm['filenames_clean2'] = df_edm['filenames_clean2'].apply(lambda x:x.upper())
df_edm.sort_values(by=['filenames_clean2'],inplace=True)
df_edm.drop(columns=['filenames_clean2'],inplace=True)
df_edm.reset_index(drop=True,inplace=True)

# # Parse the CSV output of the previous command back into a MIDI file
# midi_object = py_midicsv.csv_to_midi(csv_string)

# # Save the parsed MIDI file to disk
# with open("example_converted.mid", "wb") as output_file:
#     midi_writer = py_midicsv.FileWriter(output_file)
#     midi_writer.write(midi_object)

## Print indexes of instruments

In [6]:
for ix,song in enumerate(df_edm['csvs']):
    print(df_edm['filenames_clean'][ix]+' [index] : '+str(ix))
    for jx,row in enumerate(song):
        if 'Title' in row:
            print(jx,row)


Alan Walker - Alone.mid [index] : 0
8 3, 0, Title_t, "Lead"

336 4, 0, Title_t, "Bass"

640 5, 0, Title_t, "Drums"

878 6, 0, Title_t, "Alan Walker - Alone"

926 7, 0, Title_t, "Carlo Prato"

974 8, 0, Title_t, "cprato.com"

Alan Walker - Faded (Original Mix).mid [index] : 1
10 3, 0, Title_t, "Piano"

370 4, 0, Title_t, "Lead"

708 5, 0, Title_t, "Bass"

892 6, 0, Title_t, "Drums"

1138 7, 0, Title_t, "Alan Walker - Faded"

1194 8, 0, Title_t, "Carlo Prato"

1250 9, 0, Title_t, "cprato.com"

Alan Walker - Sing Me To Sleep.mid [index] : 2
8 3, 0, Title_t, "Lead"

312 4, 0, Title_t, "Bass"

440 5, 0, Title_t, "Drums"

1132 6, 0, Title_t, "Alan Walker - Sing Me To Sleep"

1180 7, 0, Title_t, "Carlo Prato"

1228 8, 0, Title_t, "cprato.com"

Calvin Harris - Blame.mid [index] : 3
8 3, 0, Title_t, "Lead"

344 4, 0, Title_t, "Bass"

648 5, 0, Title_t, "Drums"

886 6, 0, Title_t, "Calvin Harris - Blame"

934 7, 0, Title_t, "Carlo Prato"

982 8, 0, Title_t, "cprato.com"

Calvin Harris - My Way.m

## Print Header Info of All Songs

In [79]:
for i in range(len(instruments)):
    print(instruments[i][0])
    print('\n')

['0, 0, Header, 1, 8, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 618557\n', '2, 0, End_track\n']


['0, 0, Header, 1, 9, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 666667\n', '2, 0, Tempo, 666667\n', '2, 0, Tempo, 666667\n', '2, 0, End_track\n']


['0, 0, Header, 1, 8, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 681818\n', '2, 0, End_track\n']


['0, 0, Header, 1, 8, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 468750\n', '2, 0, End_track\n']


['0, 0, Header, 1, 9, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 500000\n', '2, 0, End_track\n']


['0, 0, Header, 1, 9, 96\n', '1, 0,

## Parse instruments for each song

In [81]:
# For each song, break it up by instruments
instruments = []
for ix,song in enumerate(df_edm['csvs']):
    holder = []
    indexes = []
    #holder.append(df_edm['filenames_clean'][ix]+' [index] : '+str(ix))
    for jx,row in enumerate(song):
        if 'Title_t' in row:
            indexes.append(jx-1)
    # Header and info before first instrument
    holder.append(song[0:indexes[0]])
    # Add each instrument for each song
    for i in range(len(indexes)-1):
        holder.append(song[indexes[i]:indexes[i+1]])
    instruments.append(holder) 

# Index of instruments that actually has notes. This is needed to get rid of instruments that doesn't have any notes
for ix,song in enumerate(instruments):
    keep_instrument_index = []
    for jx,instrument in enumerate(song[1:]):
        note_on_off = -1
        for kx,row in enumerate(instrument):
            note_on_off = row.find('Note_on')
            if note_on_off!=-1:
                keep_instrument_index.append(jx+1)
                break
    instruments[ix]=instruments[ix][:keep_instrument_index[-1]+1]

# Extract only note_on and note_off infor for melodies, bass, and drum
melody_name = ['Bassmelody','Chords','Lead','MIDI Out','Pad','Piano','Saw Chords','Second Lead','Strings',
'Voice','Voice Synth']
bass_name = ['Basses','basses','Bass','bass']
drum_name = ['Drums','drums','Drum','drum']
melody, bass, drum = [], [], []
for ix,song in enumerate(instruments):
    melody_holder, bass_holder, drum_holder = [], [], []
    for jx,instrument in enumerate(song[1:]):
        if instrument[1].split('"')[1] in bass_name:
            first_actual_note_indexes = []
            for ix,seq in enumerate(instrument):
                if ('Note_on_c' in seq) or ('Note_off_c' in seq):
                    first_actual_note_indexes.append(ix)
            bass_holder.append(instrument[first_actual_note_indexes[0]:first_actual_note_indexes[-1]+1])  
        elif instrument[1].split('"')[1] in drum_name:
            first_actual_note_indexes = []
            for ix,seq in enumerate(instrument):
                if ('Note_on_c' in seq) or ('Note_off_c' in seq):
                    first_actual_note_indexes.append(ix)
            drum_holder.append(instrument[first_actual_note_indexes[0]:first_actual_note_indexes[-1]+1])             
        else:
            first_actual_note_indexes = []
            for ix,seq in enumerate(instrument):
                if ('Note_on_c' in seq) or ('Note_off_c' in seq):
                    first_actual_note_indexes.append(ix)
            melody_holder.append(instrument[first_actual_note_indexes[0]:first_actual_note_indexes[-1]+1])
    melody.append(melody_holder)
    bass.append(bass_holder)
    drum.append(drum_holder)
    

# # Get rid of any 'Control_c','Pitch_bend_c','Program_c'
# start_end_track = []
# for ix,song in enumerate(instruments):    
#     for jx,instrument in enumerate(song):
#         remove_control_pitch_program = []
#         for kx,row in enumerate(instrument):
#             if row.split(', ')[2] not in ['Control_c','Pitch_bend_c','Program_c']:
#                 remove_control_pitch_program.append(row)
#         instruments[ix][jx] = remove_control_pitch_program  

## Play 

In [191]:
# Compile melodies across 31 songs and get unique keys being played
holder = []
for ix,song in enumerate(melody):
    for jx,instrument in enumerate(song):
        holder+=[x.split(', ') for x in instrument]  
df_all_song_melody = pd.DataFrame(holder )
# Timestamp column as integer
df_all_song_melody[1] = df_all_song_melody[1].astype('int32')
# Set the two instruments as same instrument number
df_all_song_melody[0]=3
# Set the two instruments as same instrument type
df_all_song_melody[3]=12
# Reorder df_all_song_melody by timestamp & drop duplicates
df_all_song_melody_ordered = df_all_song_melody.sort_values(by=[1]).reset_index(drop=True).copy()
df_all_song_melody_ordered.drop_duplicates(inplace=True)

# All keys across 31 songs
df_all_song_melody_ordered[4].unique()

array(['67', '55', '48', '64', '57', '71', '73', '76', '80', '74', '51',
       '41', '60', '53', '75', '68', '56', '66', '43', '69', '63', '62',
       '50', '70', '61', '78', '81', '52', '83', '72', '40', '59', '58',
       '82', '54', '65', '84', '86', '77', '42', '47', '79', '35', '45',
       '46', '87', '90', '49', '38', '39', '88', '30', '37', '89'],
      dtype=object)

In [284]:
# Process one song
song_index = 21

# Combine melodies
holder = []
for i in range(len(melody[song_index])):
    holder+=[x.split(', ') for x in melody[song_index][i]]
df_combined_instrument = pd.DataFrame(holder )
# Timestamp column as integer
df_combined_instrument[1] = df_combined_instrument[1].astype('int32')
# Set the two instruments as same instrument number
df_combined_instrument[0]=3
# Set the two instruments as same instrument type
df_combined_instrument[3]=12
# Reorder df_combined_instrument by timestamp & drop duplicates
df_combined_instrument_ordered = df_combined_instrument.sort_values(by=[1]).reset_index(drop=True).copy()
df_combined_instrument_ordered.drop_duplicates(inplace=True)
# Reconstruct ordered data into csv format to be turned into midi file
combined_instrument_ordered = []
for i in range(len(df_combined_instrument_ordered)):
    combined_instrument_ordered.append(', '.join([str(x) for x in df_combined_instrument_ordered.iloc[i].tolist()]))

# Song header and end 
song = df_edm['csvs'][song_index]
first_note_on = []
for ix,row in enumerate(song):
    if row.find('Note_on_c')!=-1:
        first_note_on.append(ix) 
song_start = song[:first_note_on[0]]    
song_end = combined_instrument_ordered[-1].split(', ')[:2]
song_end.append('End_track\n')
song_end = [', '.join(song_end)]

# Need the 'End_track' row at the end of the file so that Midi player works
csv_string = song_start+combined_instrument_ordered+song_end
play_music_csv(csv_string)

# Parse the CSV output of the previous command back into a MIDI file
midi_object = py_midicsv.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open("example_converted.mid", "wb") as output_file:
    midi_writer = py_midicsv.FileWriter(output_file)
    midi_writer.write(midi_object)

In [172]:
df_combined_instrument_ordered.head()

,0,1,2,3,4,5
0,3,0,Note_on_c,12,80,98\n
1,3,0,Note_on_c,12,76,98\n
2,3,0,Note_on_c,12,73,98\n
3,3,96,Note_on_c,12,80,98\n
4,3,96,Note_on_c,12,76,98\n


In [176]:
df_combined_instrument_ordered.groupby(by=[1,4],as_index=False).mean()[[1,4]]

,1,4
0,0,73
1,0,76
2,0,80
3,96,73
4,96,76
5,96,80
6,192,73
7,192,76
8,192,80
9,216,76


# Midi <-> Pianoroll

In [160]:
#!pip install pypianoroll
import pypianoroll

# Instrument names for look up
melody_name = ['Bassmelody','Chords','Lead','MIDI Out','Pad','Piano','Saw Chords','Second Lead','Strings',
'Voice','Voice Synth']
bass_name = ['Basses','basses','Bass','bass']
drum_name = ['Drums','drums','Drum','drum']

# Merge all tracks and Combine all songs
melody,bass,drum = [[0]*128],[[0]*128],[[0]*128]
for ix,song in enumerate(df_edm['filenames'][:]): # Switch multiple songs
#for ix,song in enumerate([df_edm['filenames'].iloc[18]]): # Play a single song
    midi_loaded = pypianoroll.Multitrack('data/midi/'+song)
    
    melody_holder = np.zeros(midi_loaded.tracks[0].pianoroll.shape)
    bass_holder   = np.zeros(midi_loaded.tracks[0].pianoroll.shape)
    drum_holder   = np.zeros(midi_loaded.tracks[0].pianoroll.shape)

    for jx,track in enumerate(midi_loaded.tracks):
        if track.name in bass_name:
            bass_holder+=track.pianoroll
        elif track.name in drum_name:
            drum_holder+=track.pianoroll    
        else:
            melody_holder+=track.pianoroll
    melody = np.append(melody,melody_holder[:-120],axis=0)
    bass = np.append(bass,bass_holder[:-120],axis=0)
    drum = np.append(drum,drum_holder[:-120],axis=0)
melody = melody[1:]
bass = bass[1:]
drum = drum[1:]

# # pypianoroll's built-in merging tracks
# midi_loaded = pypianoroll.Multitrack('data/midi/'+df_edm['filenames'][0])
# midi_loaded.merge_tracks(track_indices=list(np.arange(len(midi_loaded.tracks))[:-2]), mode='sum', program=81, is_drum=False, name='merged', remove_merged=True)
# # Save to midi file and play
# pypianoroll.write(midi_loaded,'merge_tracks.mid')
# play_music("merge_tracks.mid")

# Update blank midi file with compiled version
blank_midi = pypianoroll.Multitrack('blank.mid')
blank_midi.tempo = midi_loaded.tempo/midi_loaded.tempo*120
blank_midi.tracks[0].pianoroll = melody
blank_midi.tracks[1].pianoroll = bass
blank_midi.tracks[2].pianoroll = drum

# Save to midi file and play
pypianoroll.write(blank_midi,'compiled.mid')
play_music("compiled.mid")

Music file compiled.mid loaded!


SystemExit: 

# Midi <-> Pianoroll - Adjusted (Center Notes)

In [6]:
#!pip install pypianoroll
import pypianoroll

# Instrument names for look up
melody_name = ['Bassmelody','Chords','Lead','MIDI Out','Pad','Piano','Saw Chords','Second Lead','Strings',
'Voice','Voice Synth']
bass_name = ['Basses','basses','Bass','bass']
drum_name = ['Drums','drums','Drum','drum']

# Merge all tracks and Combine all songs
#melody,bass,drum = [[0]*128],[[0]*128],[[0]*128]
melody,bass,drum = [],[],[]

for ix,song in enumerate(df_edm['filenames'][:]): # Switch multiple songs
#for ix,song in enumerate([df_edm['filenames'].iloc[18]]): # Play a single song
    midi_loaded = pypianoroll.Multitrack('data/midi/'+song)
    
    melody_holder = np.zeros(midi_loaded.tracks[0].pianoroll.shape)
    bass_holder   = np.zeros(midi_loaded.tracks[0].pianoroll.shape)
    drum_holder   = np.zeros(midi_loaded.tracks[0].pianoroll.shape)

    for jx,track in enumerate(midi_loaded.tracks):
        if track.name in bass_name:
            bass_holder+=track.pianoroll
        elif track.name in drum_name:
            drum_holder+=track.pianoroll    
        else:
            melody_holder+=track.pianoroll
    melody.append(melody_holder[:-120])
    bass.append(bass_holder[:-120])
    drum.append(drum_holder[:-120])

# Let first song's mean note to be note to adjust other songs to. For melody and bass only.
melody_keyss = []
for seq in melody[0]:
    melody_keyss += [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
mean_note = int(np.mean(melody_keyss))

# Adjust rest of song's keys to be centered around first song's mean key. For melody and bass only.
for ix,song in enumerate(melody):
    melody_keyss = []
    for seq in song:
        melody_keyss += [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
    current_song_mean_note = int(np.mean(melody_keyss))   
    melody[ix] = np.roll(melody[ix],mean_note-current_song_mean_note)
    bass[ix] = np.roll(bass[ix],mean_note-current_song_mean_note)
    
# Concat all individual files
melody_compiled,bass_compiled,drum_compiled = melody[0],bass[0],drum[0]
for i in trange(1,len(melody)):
    melody_compiled = np.concatenate((melody_compiled,melody[i]))
    bass_compiled = np.concatenate((bass_compiled,bass[i]))
    drum_compiled = np.concatenate((drum_compiled,drum[i]))

# Update blank midi file with compiled version
blank_midi = pypianoroll.Multitrack('blank.mid')
blank_midi.tempo = midi_loaded.tempo/midi_loaded.tempo*120


blank_midi.tracks[0].pianoroll = melody_compiled
blank_midi.tracks[1].pianoroll = bass_compiled
blank_midi.tracks[2].pianoroll = drum_compiled

# Save to midi file and play
pypianoroll.write(blank_midi,'compiled_adj.mid')
play_music("compiled.mid")

100%|██████████| 28/28 [00:01<00:00,  9.24it/s]


Music file compiled_adj.mid loaded!


SystemExit: 

# 12 Semitones

In [7]:
# Create compiled midi's all 12 semitones. Then only take melody and save to semitone_adj.mid
blank_midi = pypianoroll.Multitrack('compiled_adj.mid')
blank_midi.tempo = midi_loaded.tempo/midi_loaded.tempo*120
semitone = blank_midi.tracks[0].pianoroll
semitone_compiled = blank_midi.tracks[0].pianoroll
semitone_list = [-6,-5,-4,-3,-2,-1,1,2,3,4,5,6]
for i in trange(len(semitone_list)):
    semitone_compiled = np.concatenate((semitone_compiled,np.roll(semitone,semitone_list[i])))
blank_midi.tracks[0].pianoroll = semitone_compiled
blank_midi.tracks.pop(1)
blank_midi.tracks.pop(1)
pypianoroll.write(blank_midi,'semitone_adj.mid')
#play_music("semitone.mid")

100%|██████████| 12/12 [00:00<00:00, 21.86it/s]


Music file semitone_adj.mid loaded!


SystemExit: 

## Mapping for Char-RNN

In [5]:
# Find index of keys being played for each sequence so that it can be mapped
melody_keys = []
for seq in melody:
    seq_keys = []
    seq_keys = [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
    if len(seq_keys)==0:
        seq_keys = [1] 
    melody_keys.append(seq_keys)    
bass_keys = []
for seq in bass:
    seq_keys = []
    seq_keys = [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
    if len(seq_keys)==0:
        seq_keys = [1]
    bass_keys.append(seq_keys)    
drum_keys = []
for seq in drum:
    seq_keys = []
    seq_keys = [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
    if len(seq_keys)==0:
        seq_keys = [1]
    drum_keys.append(seq_keys)
    
# Key to char and char to key mapping
standard_ascii = [chr(i) for i in range(128)]
key_to_char_list = standard_ascii[32:-1]
key_to_char_list.remove('\\')
key_to_char_dict = defaultdict()
char_to_key_dict = defaultdict()
for ix,char in enumerate(key_to_char_list):
    key_to_char_dict[ix]=char
    char_to_key_dict[char]=ix
    
# Convert keys to characters using mapping
melody_chars = []
for i,seq in enumerate(melody_keys):
    holder = []
    for j,key in enumerate(seq):
        holder+=key_to_char_dict[key]
    melody_chars.append(holder)

In [497]:
# # For Andrej Karpathy's Char-RNN

# # Process one more time to be fed into char-RNN
# melody_chars_combined = ''
# for char in melody_chars:
#     melody_chars_combined+=''.join(char)+' '  

# # Save file to a txt file
# file = open('/Users/matthewlee/Desktop/Metis/char-rnn/data/tinyshakespeare/input.txt','w') 
# #file = open('input.txt','w')  
# file.write(melody_chars_combined) 
# file.close() 

# # Load txt file
# file_object  = open('input.txt','r')

In [466]:
# # Identify all keys present across melody & bass & drum. ***1 is empty sequence.
# melody_keyss = []
# for seq in melody:
#     seq_keys = []
#     seq_keys = [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
#     if len(seq_keys)==0:
#         seq_keys = [1] 
#     melody_keyss+=seq_keys
# bass_keyss = []
# for seq in bass:
#     seq_keys = []
#     seq_keys = [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
#     if len(seq_keys)==0:
#         seq_keys = [1]
#     bass_keyss+=seq_keys
    
# drum_keyss = []
# for seq in drum:
#     seq_keys = []
#     seq_keys = [ ix for (ix,x) in enumerate(seq) if x >= 10 ]
#     if len(seq_keys)==0:
#         seq_keys = [1]
#     drum_keyss+=seq_keys
# list(set(melody_keyss+bass_keyss+drum_keyss))

[1,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 86,
 87,
 88,
 89,
 90]

# Follow Medium Article

In [230]:
# #!pip install music21
from music21 import converter, instrument, note, chord,stream
# notes = []
# for file in glob.glob("semitone_adj.mid"):
#     midi = converter.parse(file)
#     notes_to_parse = None
#     parts = instrument.partitionByInstrument(midi)
#     if parts: # file has instrument parts
#         notes_to_parse = parts.parts[0].recurse()
#     else: # file has notes in a flat structure
#         notes_to_parse = midi.flat.notes
#     for element in notes_to_parse:
#         if isinstance(element, note.Note):
#             notes.append(str(element.pitch))
#         elif isinstance(element, chord.Chord):
#             notes.append('.'.join(str(n) for n in element.normalOrder))
# pickle.dump( notes, open( "notes.pkl", "wb" ) )
notes = pickle.load( open( "notes.pkl", "rb" ) )

# Modeling

## Train

In [41]:
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,Flatten,GlobalAveragePooling1D,AveragePooling1D,GlobalMaxPooling1D,BatchNormalization,Activation,Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [42]:
sequence_length = 70
n_vocab = len(list(set(notes)))
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

In [43]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 576/6591 [=>............................] - ETA: 6:17 - loss: 4.7789

KeyboardInterrupt: 

## Load Weight and Generate Melody

In [235]:
# Load the weights
model.load_weights('weights-improvement-66-0.4656-bigger.hdf5')

# Don't always choose the most likely prediction
def sample(preds, temperature=0.9):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Generate new song
start = np.random.randint(0, len(network_input)-1)
print(start)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = list(network_input[start])
prediction_output = []
# generate 300 notes
for note_index in trange(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    #index = np.argmax(prediction)
    index = sample(prediction[0])
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

  0%|          | 1/500 [00:00<01:06,  7.52it/s]

35666


100%|██████████| 500/500 [00:52<00:00,  7.99it/s]


In [236]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.4

In [237]:
# Play the song
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')
play_music('test_output.mid')

Music file test_output.mid loaded!


In [31]:
play_music('compiled.mid')

Music file compiled.mid loaded!


SystemExit: 

# Pitch Tracking

In [85]:
44100/2

22050.0

In [93]:
22050/2

11025.0

In [216]:
# Record Audio

#!conda install -c anaconda pyaudio --yes
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1
fs =  22050  # Record at 44100 samples per second
seconds = 3
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


In [217]:
# Play Audio

#!conda install -c conda-forge pydub --yes
from pydub import AudioSegment
from pydub.playback import play

song = AudioSegment.from_wav("output.wav")
play(song)

In [224]:
!audio-to-midi ./output.wav --time-window 400 --activation-level 0.7 --single-note --condense
play_music('output.wav.mid')

Converting: ./output.wav
|================================================================================| 100.00%
Music file output.wav.mid loaded!


# Download Youtube

In [156]:
# Use below command in the terminal to download youtube file as wav file.
#!python YouTube_to_WAV.py https://www.youtube.com/watch?v=xgs-laNZ0SE&t=218s

# Midi -> Wav (Doesn't work atm)

In [17]:
# #!pip install midi2audio --y
from midi2audio import FluidSynth

# using the default sound font in 44100 Hz sample rate
fs = FluidSynth()
fs.midi_to_audio('Alan Walker - Alone  (midi by Carlo Prato) (www.cprato.com) copy.mid','output.wav')
# for ix,filename in enumerate(filenames):
#     fs.midi_to_audio('data/midi/'+filename, 'data/wav/'+df_edm['filenames_clean'][ix][:-4]+'.wav')


# Play ABC Notation

In [62]:
# ABC Notation Player
import pysynth as ps

# Make test ABC file
test = (
  ('g#', 4),  ('g#', 4), ('g5', 4),
  ('g5*', 4), ('r', 4), ('e5', 16),
  ('f5', 16),  ('e5', 16),  ('d5', 16),
  ('e5*', 4) 
)
ps.make_wav(test, fn = "test.wav")

#Play wav file
ipd.Audio('test.wav') # load a local WAV file

Piano key frequencies (for equal temperament):
Key number	Scientific name	Frequency (Hz)
         1	             A0	         27.50
         2	            A#0	         29.14
         3	             B0	         30.87
         4	             C1	         32.70
         5	            C#1	         34.65
         6	             D1	         36.71
         7	            D#1	         38.89
         8	             E1	         41.20
         9	             F1	         43.65
        10	            F#1	         46.25
        11	             G1	         49.00
        12	            G#1	         51.91
        13	             A1	         55.00
        14	            A#1	         58.27
        15	             B1	         61.74
        16	             C2	         65.41
        17	            C#2	         69.30
        18	             D2	         73.42
        19	            D#2	         77.78
        20	             E2	         82.41
        21	             F2	         87.31
        22	            F#2	  